In [1]:
import functools
import itertools
import pickle
import string
from pathlib import Path

import cytoolz as tlz
import geopandas as gpd
import mpl_toolkits.axes_grid1 as mgrid
import numpy as np
import utils
import xarray as xr
import xskillscore as xs
from matplotlib import cm
from matplotlib import pyplot as plt
from matplotlib import tri
from scipy import interpolate

In [2]:
CLASSES = {
    "C1": "w/o Estuary",
    "C2": "Triangular Estuary",
    "C3": "Trapezoidal Estuary",
}

SAVE_KWDS = {"bbox_inches": "tight", "dpi": 300, "facecolor": "w"}

LABELS = {
    "S1": "Ref",
    "S2_1": "R20",
    "S2_2": "R30",
    "S4_1": "D90",
    "S4_2": "D570",
    "S5_1": "S07",
    "S5_2": "S31",
}

In [3]:
def dflow_triangulate(nx, ny, config):
    triang = tri.Triangulation(nx, ny)
    x = nx[triang.triangles].mean(axis=1)
    y = ny[triang.triangles].mean(axis=1)

    m = []
    if config["class"] == 1:
        m.append(np.where((x > config["x_o1"]) & (x < config["x_r1"]) & (y > config["y_o"]), 1, 0))
        m.append(np.where((x > config["x_r2"]) & (x < config["x_o2"]) & (y > config["y_o"]), 1, 0))
    else:
        if (config["x_b3"] - config["x_b1"]) < 1e-2:
            s_w, s_e = 0e0, 0e0
        else:
            s_w = (config["y_r"] - config["y_b"]) / (config["x_b3"] - config["x_b1"])
            s_e = (config["y_r"] - config["y_b"]) / (config["x_b4"] - config["x_b2"])

        m.append(np.where((x > config["x_o1"]) & (x < config["x_b1"]) & (y > config["y_o"]), 1, 0))
        m.append(np.where((x > config["x_b2"]) & (x < config["x_o2"]) & (y > config["y_o"]), 1, 0))
        m.append(np.where((x > config["x_b1"]) & (x < config["x_b3"]) & (y > config["y_b"]), 1, 0))
        m.append(np.where((x > config["x_b4"]) & (x < config["x_b2"]) & (y > config["y_b"]), 1, 0))
        m.append(np.where((x > config["x_b3"]) & (x < config["x_r1"]) & (y > config["y_b"]), 1, 0))
        m.append(np.where((x > config["x_r2"]) & (x < config["x_b4"]) & (y > config["y_b"]), 1, 0))
        m.append(
            np.where(
                (x > config["x_b1"])
                & (x < config["x_b3"])
                & (y > config["y_b"] + s_w * (x - config["x_b1"])),
                1,
                0,
            )
        )
        m.append(
            np.where(
                (x > config["x_b4"])
                & (x < config["x_b2"])
                & (y > config["y_b"] + s_e * (x - config["x_b2"])),
                1,
                0,
            )
        )

    mask = m[0]
    for i in m[1:]:
        mask = mask + i
    mask[mask > 1] = 1
    triang.set_mask(mask)
    return triang

In [4]:
def gridded_mae_test(ref_path, case1_path, case2_path, variable, metric="mae", alpha=0.05):
    wl_ref = xr.open_dataset(ref_path, chunks="auto")[variable]
    config_ref = utils.read_config(ref_path.parent.joinpath("inputs.txt"))

    nx = wl_ref.mesh2d_face_x.values
    ny = wl_ref.mesh2d_face_y.values
    triang = dflow_triangulate(nx, ny, config_ref)

    wl_case1 = xr.open_dataset(case1_path, chunks="auto")[variable]
    wl_case2 = xr.open_dataset(case2_path, chunks="auto")[variable]

    dims = {da.time.shape[0] for da in [wl_ref, wl_case1, wl_case2]}
    if len(dims) > 1:
        dmin = min(list(dims))
        wl_ref, wl_case1, wl_case = (
            da.isel(time=slice(0, dmin)) for da in [wl_ref, wl_case1, wl_case2]
        )

    _, diff, hwci = xs.halfwidth_ci_test(wl_case1, wl_case2, wl_ref, metric, alpha=alpha)
    score_sig = np.sign(diff) * (np.abs(diff) - hwci)

    cname = ref_path.parent.name.split("_")[0]
    return cname, (triang, score_sig)

In [5]:
def plot_dflow(ax, triang, da, vmin, vmax, cmap="coolwarm_r"):
    nx = da.mesh2d_face_x.values
    ny = da.mesh2d_face_y.values
    wdt = interpolate.griddata((nx, ny), da.values, (triang.x, triang.y), method="linear")

    norm = cm.colors.Normalize(vmax=vmax, vmin=vmin)
    levels = np.linspace(vmin, vmax, 10)
    cmap_lvls = cm.get_cmap(getattr(cm, cmap), len(levels) - 1)

    tcf = ax.tricontourf(
        triang,
        wdt,
        levels,
        cmap=cmap_lvls,
        norm=norm,
    )
    ax.tricontour(triang, wdt, tcf.levels, colors="k")
    ax.grid(color="g", linestyle="-.", linewidth=0.3)
    ax.set_axisbelow(True)

    xmin, xmax = 0e3, 50e3

    xticks = ax.get_xticks()
    xlabels = np.linspace(xmin * 1e-3, xmax * 1e-3, len(xticks)).astype("int")
    ax.set_xticks(xticks)
    ax.set_xticklabels(xlabels)

    yticks = np.arange(0, 1120, 120).astype("int")
    ax.set_yticks(yticks * 1e3)
    ax.set_yticklabels(yticks)
    ax.set_ylim(0, yticks[-1] * 1e3)

    ax.margins(0)

    x1, x2, y1, y2 = 18.0e3, 32.0e3, 80.0e3, 330.0e3
    axins = ax.inset_axes([0.58, 0.35, 0.4, 0.4])
    tcf_zoom = axins.tricontourf(
        triang,
        wdt,
        levels,
        cmap=cmap_lvls,
        norm=norm,
    )
    axins.tricontour(triang, wdt, tcf_zoom.levels, colors="k")
    axins.set_xlim(x1, x2)
    axins.set_ylim(y1, y2)
    axins.set_xticklabels("")
    axins.set_yticklabels("")

    ax.indicate_inset_zoom(axins, edgecolor="k")
    return tcf

In [6]:
def plot_mae_diff(variables, params, out_root, img_root, ci_alpha=95, cmap="coolwarm_r"):
    for v, u in variables.items():
        fig, ax_rows = plt.subplots(
            3,
            3,
            figsize=(11, 11),
            dpi=300,
            sharey=True,
            sharex=True,
            gridspec_kw={"hspace": 0.05, "wspace": 0.05},
        )

        axs_arr = np.array(ax_rows)
        alphabest = [f"({s})" for s in string.ascii_lowercase[: len(ax_rows) * len(ax_rows[0])]]
        fig_label = list(tlz.partition(len(ax_rows[0]), alphabest))
        for axs, (p, (s1, s2)) in zip(ax_rows, params.items()):
            with open(Path(out_root, f"mae_diff_{v}_{p.lower()}.pkl"), "rb") as f:
                data = pickle.load(f)

            vmax = max(da.map_blocks(np.fabs).max().values for _, (_, da) in data.items())
            vmin = -vmax
            for ax, (_, (triang, da)) in zip(axs, data.items()):
                i, j = np.argwhere(axs_arr == ax)[0]
                tcf = plot_dflow(ax, triang, da, vmin, vmax, cmap=cmap)
                ax.text(
                    0.02,
                    0.97,
                    fig_label[i][j],
                    size=12,
                    horizontalalignment="left",
                    verticalalignment="top",
                    transform=ax.transAxes,
                )

            axs[0].set_ylabel(f"{s2} vs. {s1}\ny (km)")

            divider = mgrid.make_axes_locatable(axs[-1])
            cax = divider.append_axes("right", size="3%", pad=0.1)
            fig.colorbar(
                tcf,
                format="%.3f",
                cax=cax,
                label=fr"$SD$ ({u})",
            )
        [ax.set_title(CLASSES[c], loc="center", y=1.0) for ax, c in zip(ax_rows[0], data)]
        [ax.set_xlabel("x (km)") for ax in ax_rows[-1]]
        img_path = Path(img_root, f"mae_diff_{v}.tiff")
        fig.savefig(img_path, **SAVE_KWDS)
        plt.close("all")

In [7]:
root = Path("..", "data")
inp_path = Path(root, "inputs")
out_path = Path(root, "outputs")
img_path = Path(root, "figures")
sim_path = Path(Path.home(), "repos", "dev", "results")

params = {
    "Discharge": ("D90", "D570"),
    "Roughness": ("R20", "R30"),
    "Surge": ("S07", "S31"),
}
variables = {"mesh2d_s1": "m", "mesh2d_ucmag": "m/s"}
class_list = [1, 2, 3]

stations = gpd.read_feather(Path(out_path, "gulf_east_wl_stations.feather")).set_index("id")
lat = stations.loc["8534720"].lat

In [8]:
for v, p in itertools.product(variables, params):
    pkl_name = Path(out_path, f"mae_diff_{v}_{p.lower()}.pkl")
    if not pkl_name.exists():
        f_ref = functools.partial(utils.get_path, "Ref", root=sim_path)
        f_s1 = functools.partial(utils.get_path, params[p][0], root=sim_path)
        f_s2 = functools.partial(utils.get_path, params[p][1], root=sim_path)
        data = dict(gridded_mae_test(f_ref(c), f_s1(c), f_s2(c), v) for c in class_list)

        with open(pkl_name, "wb") as f:
            pickle.dump(data, f)

In [9]:
plot_mae_diff(variables, params, out_path, img_path)